In [5]:
!pip install beautifulsoup4
!pip install lxml
!conda install -c conda-forge folium=0.5.0 --yes
import pandas as pd 
import numpy as np 
import requests 
import random 
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from IPython.display import Image 
from IPython.core.display import HTML
from IPython.display import display_html
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.8-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    certifi-2021.5.30          |   py38h578d9bd_0         141 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.8             |           2_cp38           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    widgetsnbextension-3.5.1   |   py38h578d9bd_4         1.8 MB  conda-forge
    ------------------------------------------------------------
          

In [6]:
zipatlas_nyc_data = requests.get('http://zipatlas.com/us/ny/new-york/zip-code-comparison/median-household-income.htm').text
soup=BeautifulSoup(zipatlas_nyc_data,'lxml')
#In the webpage Table 13 has the data

table_rows = soup.find_all('table')[13].tr.next_siblings

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=['Number','Zip_Code','Location','City','Population','Avg_Income','Ratings'])
df.head()

,Number,Zip_Code,Location,City,Population,Avg_Income,Ratings
0,1.,10007,"40.713973, -74.008336","New York, New York","3,522","$112,947.00",#132
1,2.,10280,"40.708501, -74.016828","New York, New York","6,614","$108,536.00",#157
2,3.,10162,"40.769298, -73.949942","New York, New York","1,726","$108,416.00",#158
3,4.,10004,"40.691870, -74.014109","New York, New York","1,225","$101,868.00",#238
4,5.,10069,"40.775551, -73.989689","New York, New York","1,403","$88,091.00",#507


In [7]:
# Spilitting Loaction into Latitude and Longitude information seperately
df[['lat','lng']] = df["Location"].str.split(",",expand=True)
#Get the neighborhood's name.
df.head()

In [8]:
#Generating Map for Manhattan, we need geograpical coordinate of Manhattan 
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan, NY are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan, NY are 40.7896239, -73.9598939.


In [13]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for Location, zip_code, Avg_Income, Population in zip(df['Location'], df['Zip_Code'], df['Avg_Income'], df['Population']):
    label = '{}, {},{}'.format('zipcode:'+ zip_code, 'Avg Income:'+ Avg_Income,'Population:'+ Population)
    label = folium.Popup(label, parse_html=True)
    lat = float(Location.split(",")[0])
    long = float(Location.split(",")[1])
    folium.CircleMarker(
        [lat, long],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [14]:
# The code was removed by Watson Studio for sharing.

In [16]:
VERSION = '20180605' # Foursquare API version
LIMIT = 300 # A default Foursquare API limit value
radius = 1000
dataframe_empty = pd.DataFrame()
for index, row in df.iterrows():
  lat = row['lat']
  lng = row['lng']
  zipcode = row['Zip_Code']

  # Use category_id = '4d4b7105d754a06374d81259' for restaurent
  url = 'https://api.foursquare.com/v2/venues/search?&categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
  results = requests.get(url).json()
  # assign relevant part of JSON to venues
  venues = results['response']['venues']
  # tranform venues into a dataframe
  dataframe_2 = pd.json_normalize(venues)
  dataframe_2['Zip_Code'] = zipcode
  dataframe_empty = dataframe_empty.append(dataframe_2)           
              
dataframe_empty.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.formattedAddress,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id,Zip_Code
0,5d5f24ec09484500079aee00,Los Tacos No. 1,"[{'id': '4bf58dd8d48988d151941735', 'name': 'T...",v-1633843043,False,136 Church St,btw Murray & Warren,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng':...",...,"[136 Church St (btw Murray & Warren), New York...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10007
1,4c154c9a77cea593c401d260,Takahachi Bakery,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1633843043,False,25 Murray St,at Church St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.71365284530189...",...,"[25 Murray St (at Church St), New York, NY 100...",Tribeca,2515459,https://www.seamless.com/menu/takahachi-bakery...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,85495566,10007
2,5f4bf912e202011fc38b19b1,Liberty Bistro,"[{'id': '52e81612bcbc57f1066b79f1', 'name': 'B...",v-1633843043,False,225 Liberty St,NaN,40.712642,-74.016064,"[{'label': 'display', 'lat': 40.712642, 'lng':...",...,"[225 Liberty St, New York, NY 10281, United St...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10007
3,4a56a32ff964a52090b51fe3,Jubilee Marketplace,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",v-1633843043,False,99 John St,at Gold St,40.708241,-74.006468,"[{'label': 'display', 'lat': 40.70824059448364...",...,"[99 John St (at Gold St), New York, NY 10038, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10007
4,49ccd495f964a52091591fe3,Kaffe 1668,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1633843043,False,275 Greenwich St,at Murray St,40.715045,-74.011509,"[{'label': 'display', 'lat': 40.71504512558996...",...,"[275 Greenwich St (at Murray St), New York, NY...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,43231093,10007


In [17]:
dataframe_empty.shape

(3200, 26)

In [18]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories','Zip_Code'] + [col for col in dataframe_empty.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe_empty.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,Zip_Code,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Los Tacos No. 1,Taco Place,10007,136 Church St,btw Murray & Warren,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng':...",48,10007,US,New York,NY,United States,"[136 Church St (btw Murray & Warren), New York...",NaN,5d5f24ec09484500079aee00
1,Takahachi Bakery,Bakery,10007,25 Murray St,at Church St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.71365284530189...",53,10007,US,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 100...",Tribeca,4c154c9a77cea593c401d260
2,Liberty Bistro,Bistro,10007,225 Liberty St,NaN,40.712642,-74.016064,"[{'label': 'display', 'lat': 40.712642, 'lng':...",668,10281,US,New York,NY,United States,"[225 Liberty St, New York, NY 10281, United St...",NaN,5f4bf912e202011fc38b19b1
3,Jubilee Marketplace,Grocery Store,10007,99 John St,at Gold St,40.708241,-74.006468,"[{'label': 'display', 'lat': 40.70824059448364...",657,10038,US,New York,NY,United States,"[99 John St (at Gold St), New York, NY 10038, ...",NaN,4a56a32ff964a52090b51fe3
4,Kaffe 1668,Coffee Shop,10007,275 Greenwich St,at Murray St,40.715045,-74.011509,"[{'label': 'display', 'lat': 40.71504512558996...",293,10007,US,New York,NY,United States,"[275 Greenwich St (at Murray St), New York, NY...",NaN,49ccd495f964a52091591fe3
5,Laughing Man Coffee & Tea,Coffee Shop,10007,184 Duane St,btwn Greenwich & Hudson St.,40.717394,-74.010103,"[{'label': 'display', 'lat': 40.7173944529165,...",409,10013,US,New York,NY,United States,"[184 Duane St (btwn Greenwich & Hudson St.), N...",Tribeca,4ea0afbf9adf1e334e4cc0e6
6,1803,Cajun / Creole Restaurant,10007,82 Reade St,at Church St,40.715418,-74.007425,"[{'label': 'display', 'lat': 40.71541804978279...",178,10007,US,New York,NY,United States,"[82 Reade St (at Church St), New York, NY 1000...",NaN,5977d66b64c8e14048a7af6d
7,Kenn's Broome Street Bar,Bar,10007,363 W Broadway,at Broome St,40.723316,-74.003200,"[{'label': 'display', 'lat': 40.72331594075375...",1126,10013,US,New York,NY,United States,"[363 W Broadway (at Broome St), New York, NY 1...",NaN,3fd66200f964a52094e61ee3
8,Joe’s Pizza,Pizza Place,10007,124 Fulton St,at Nassau St,40.710178,-74.007769,"[{'label': 'entrance', 'lat': 40.710123, 'lng'...",425,10038,US,New York,NY,United States,"[124 Fulton St (at Nassau St), New York, NY 10...",NaN,5c6f03f30802d4002c16884c
9,Clinton Hall,Beer Garden,10007,90 Washington St,at Rector St,40.708219,-74.014460,"[{'label': 'display', 'lat': 40.70821932244674...",822,10006,US,New York,NY,United States,"[90 Washington St (at Rector St), New York, NY...",NaN,51b77ea1454a57610205b352


In [19]:
dataframe_filtered['categories'].unique()

array(['Taco Place', 'Bakery', 'Coffee Shop', 'Bistro', 'Grocery Store',
       'Food Court', 'American Restaurant', 'Dim Sum Restaurant',
       'Bagel Shop', 'Italian Restaurant', 'Cha Chaan Teng',
       'French Restaurant', 'Seafood Restaurant', 'Chinese Restaurant',
       'Mexican Restaurant', 'Gourmet Shop', 'Pizza Place', 'Café',
       'Shanghai Restaurant', 'Restaurant', 'New American Restaurant',
       'Dessert Shop', 'Cajun / Creole Restaurant', 'Flea Market',
       'Asian Restaurant', 'Beer Garden', 'Ice Cream Shop', 'Pub',
       'Wine Bar', 'Cocktail Bar', 'Burger Joint', 'Bar',
       'Bubble Tea Shop', 'Salad Place', 'Sandwich Place',
       'Korean Restaurant', 'Japanese Restaurant', 'Donut Shop',
       'Mediterranean Restaurant', 'Event Space', 'Greek Restaurant',
       'Fast Food Restaurant', 'Deli / Bodega', 'Vietnamese Restaurant',
       'Sushi Restaurant', 'Thai Restaurant', 'Diner', 'Gastropub',
       'Spanish Restaurant', 'Noodle House', 'Tapas Restaurant

In [19]:
# Filtering out 'Bakery','Grocery Store', 'Flea Market','Furniture / Home Store', 'Convenience Store', 'Gym / Fitness Center', 'Chocolate Shop' from the list
data = dataframe_filtered.loc[dataframe_filtered['categories'] != ('Bakery','Grocery Store', 'Flea Market','Furniture / Home Store', 'Convenience Store', 'Gym / Fitness Center', 'Chocolate Shop')]

In [20]:
# Count of venues (restaurent) per zip code
data_final = data.groupby("Zip_Code")["name"].count()
data_final.head(10)

Zip_Code
10001    50
10002    50
10003    50
10004    50
10005    50
10006    50
10007    50
10009    50
10010    50
10011    50
Name: name, dtype: int64

In [21]:
# Merge count of resturants with the actual data
merged_inner = pd.merge(left=df, right=data_final, left_on='Zip_Code', right_on='Zip_Code')
merged_inner.rename(columns={"name" : "Restaurant_count"}, errors="raise")
merged_inner.head()

,Number,Zip_Code,Location,City,Population,Avg_Income,Ratings,lat,lng,name
0,1.,10007,"40.713973, -74.008336","New York, New York","3,522","$112,947.00",#132,40.713973,-74.008336,50
1,2.,10280,"40.708501, -74.016828","New York, New York","6,614","$108,536.00",#157,40.708501,-74.016828,50
2,3.,10162,"40.769298, -73.949942","New York, New York","1,726","$108,416.00",#158,40.769298,-73.949942,50
3,4.,10004,"40.691870, -74.014109","New York, New York","1,225","$101,868.00",#238,40.691870,-74.014109,50
4,5.,10069,"40.775551, -73.989689","New York, New York","1,403","$88,091.00",#507,40.775551,-73.989689,50


In [22]:
#merged_inner['Avg_Income'] = merged_inner['Avg_Income'].str.replace('$','', regex=True)
merged_inner['Avg_Income'] = merged_inner['Avg_Income'].map(lambda x: x.lstrip('$').rstrip(''))
merged_inner['Avg_Income'] = merged_inner['Avg_Income'].str.replace(',','',regex=True)
merged_inner['Population'] = merged_inner['Population'].str.replace(',','',regex=True)
merged_inner['Population'] = merged_inner['Population'].astype(int)
merged_inner['Avg_Income'] = merged_inner['Avg_Income'].str.replace('.00','', regex=True)
merged_inner.head(10)

,Number,Zip_Code,Location,City,Population,Avg_Income,Ratings,lat,lng,name
0,1.,10007,"40.713973, -74.008336","New York, New York",3522,112947,#132,40.713973,-74.008336,50
1,2.,10280,"40.708501, -74.016828","New York, New York",6614,108536,#157,40.708501,-74.016828,50
2,3.,10162,"40.769298, -73.949942","New York, New York",1726,108416,#158,40.769298,-73.949942,50
3,4.,10004,"40.691870, -74.014109","New York, New York",1225,101868,#238,40.691870,-74.014109,50
4,5.,10069,"40.775551, -73.989689","New York, New York",1403,88091,#507,40.775551,-73.989689,50
5,6.,10282,"40.717017, -74.015566","New York, New York",1574,86066,#591,40.717017,-74.015566,50
6,7.,10006,"40.707970, -74.013610","New York, New York",1447,81334,#766,40.707970,-74.013610,50
7,8.,10022,"40.758615, -73.967752","New York, New York",30642,80406,#820,40.758615,-73.967752,50
8,9.,10005,"40.705390, -74.008988","New York, New York",884,79517,#860,40.705390,-74.008988,50
9,10.,10024,"40.786307, -73.970694","New York, New York",61414,78066,#935,40.786307,-73.970694,50


In [73]:
print('The best zip code to for opening a restaurent is:')
merged_inner.sort_values(by=['name','Population','Avg_Income'], ascending=False).head(1)

The best zip code to for opening a restaurent is:


,Number,Zip_Code,Location,City,Population,Avg_Income,Ratings,lat,lng,name
11,12.,10021,"40.769742, -73.961653","New York, New York",102078,75472,"#1,082",40.769742,-73.961653,50


In [36]:
venue_for_10021 = dataframe_filtered[dataframe_filtered['Zip_Code'] == '10021']

In [37]:
venue_for_10021.head(5)

,name,categories,Zip_Code,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,EJ's Luncheonette,Diner,10021,1271 3rd Ave,at E 73rd St,40.770459,-73.959843,"[{'label': 'display', 'lat': 40.770459, 'lng':...",172,10021,US,New York,NY,United States,"[1271 3rd Ave (at E 73rd St), New York, NY 100...",NaN,3fd66200f964a520dbe71ee3
1,Tacombi,Taco Place,10021,202 E 70th St,NaN,40.768204,-73.961321,"[{'label': 'display', 'lat': 40.768204, 'lng':...",173,10021,US,New York,NY,United States,"[202 E 70th St, New York, NY 10021, United Sta...",NaN,5fe3be54293b492864d91a2a
2,J.G. Melon,Burger Joint,10021,1291 3rd Ave,at E 74th St,40.771050,-73.959310,"[{'label': 'display', 'lat': 40.77105, 'lng': ...",245,10021,US,New York,NY,United States,"[1291 3rd Ave (at E 74th St), New York, NY 100...",NaN,3fd66200f964a52044ea1ee3
3,Two Wheels,Vietnamese Restaurant,10021,1382 2nd Ave,NaN,40.768101,-73.958535,"[{'label': 'display', 'lat': 40.768101, 'lng':...",320,10021,US,New York,NY,United States,"[1382 2nd Ave, New York, NY 10021, United States]",NaN,60551c9f9000240b6a7b1e1f
4,Piccola Cucina Uptown,Italian Restaurant,10021,106 E 60th St,NaN,40.763234,-73.968982,"[{'label': 'display', 'lat': 40.763234, 'lng':...",952,10022,US,New York,NY,United States,"[106 E 60th St, New York, NY 10022, United Sta...",NaN,5ec96f261803890008679e3b


In [38]:
# create map of New York using latitude and longitude values
map_newyork_10021 = folium.Map(location=[40.769218, -73.957901	], zoom_start=15)

# add markers to map
for name,lat,lng, address in zip(venue_for_10021['name'], venue_for_10021['lat'], venue_for_10021['lng'], venue_for_10021['address']):
    label = '{}, {}'.format('name:'+ name, 'Address:'+ address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(long)],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork_10021)  
    
map_newyork_10021

In [39]:
venue_for_10021_grouped = venue_for_10021.groupby('categories').mean().reset_index()
venue_for_10021_grouped
venue_for_10021_grouped_org = venue_for_10021_grouped.copy()

In [40]:
# set number of clusters
kclusters = 5

venue_for_10021_grouped = venue_for_10021_grouped.drop('categories', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venue_for_10021_grouped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 0, 0, 0, 3, 3, 1, 3, 3], dtype=int32)

In [41]:
# add clustering labels
venue_for_10021_grouped_org.insert(0, 'Cluster Labels', kmeans.labels_)
venue_for_10021_grouped_org.head() # check the last columns!

,Cluster Labels,categories,lat,lng,distance
0,3,American Restaurant,40.773871,-73.962083,734.500000
1,1,Bagel Shop,40.764294,-73.958518,661.000000
2,0,Bakery,40.774291,-73.961541,927.666667
3,0,Bar,40.776475,-73.952585,1070.000000
4,0,Beer Garden,40.760907,-73.963058,990.000000


In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(venue_for_10021_grouped_org['lat'], venue_for_10021_grouped_org['lng'], venue_for_10021_grouped_org['categories'], venue_for_10021_grouped_org['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [55]:
for i in range(kclusters):
 print ('Examining cluster: '+ str(i))
 print(venue_for_10021_grouped_org.loc[venue_for_10021_grouped_org['Cluster Labels'] == i, venue_for_10021_grouped_org.columns[[1] + list(range(5, venue_for_10021_grouped_org.shape[1]))]])

Examining cluster: 0
                  categories
2                     Bakery
3                        Bar
4                Beer Garden
12                 Gastropub
15  Mediterranean Restaurant
19               Salad Place
Examining cluster: 1
            categories
1           Bagel Shop
7         Cocktail Bar
11   French Restaurant
16        Noodle House
17         Pizza Place
20  Spanish Restaurant
21    Sushi Restaurant
23    Tapas Restaurant
Examining cluster: 2
               categories
18             Restaurant
24        Thai Restaurant
25  Vietnamese Restaurant
Examining cluster: 3
             categories
0   American Restaurant
5          Burger Joint
6                  Café
8           Coffee Shop
9          Dessert Shop
13       Ice Cream Shop
14   Italian Restaurant
Examining cluster: 4
    categories
10       Diner
22  Taco Place


In [61]:
final_cluster_data = venue_for_10021_grouped_org[venue_for_10021_grouped_org['Cluster Labels'] ==1]
final_cluster_data.head()

,Cluster Labels,categories,lat,lng,distance
1,1,Bagel Shop,40.764294,-73.958518,661.0
7,1,Cocktail Bar,40.770082,-73.954978,564.0
11,1,French Restaurant,40.766829,-73.967579,595.0
16,1,Noodle House,40.772233,-73.954447,667.0
17,1,Pizza Place,40.770977,-73.959851,586.5


In [65]:
max_lat = final_cluster_data['lat'].max()
max_lng = final_cluster_data['lng'].max()
min_lat = final_cluster_data['lat'].min()
min_lng = final_cluster_data['lng'].min()
print('{}, {}, {}, {}'.format('Maximum lattitude:'+ str(max_lat), 'Maximum Longitude:'+ str(max_lng), 'Minimum lattitude:'+ str(min_lat), 'Minimum Longitude:'+ str(min_lng)))

Maximum lattitude:40.773786, Maximum Longitude:-73.95444729258693, Minimum lattitude:40.76429402165898, Minimum Longitude:-73.96757946152178


In [72]:
m = folium.Map(location=[max_lat, min_lng], zoom_start=15)

loc = [(min_lat, min_lng),
       (max_lat, max_lng)]

folium.PolyLine(loc,
                color='red',
                weight=15,
                opacity=0.8).add_to(m)
m